In [1]:
import os
import pandas as pd

In [2]:
for root, dirs,files in os.walk("tmp/", topdown=True):
    txts=[]
    for name in files:
        txts.append(os.path.join(root, name))
    break

In [3]:
final=[]
for txt in txts:
    f = open(txt,'r',encoding = "ISO-8859-1")
    number = f.name.replace('tmp/','')
    number = number.replace('.txt','')
    f.readline()
    x = f.readline().strip()
    x = x.replace('[','\"[')
    x = x.replace(']',']\"')
    final.append(number+','+x[:-1])

In [4]:
final

['74,1,0.01,adam,avg,1000,"[-10.0, 1000.0, -1.0]",None,relu,10,200,0.1,0.99,100,10,True,10,False']

In [5]:
with open('ff.csv','w') as f:
    for txt in final:
        f.write(txt+'\n')

In [6]:
data= pd.read_csv('ff.csv',header=None)

In [7]:
data.sort_values(by=[0],inplace=True)

In [8]:
data['20'] = data[data.columns[6]]
data.drop(data.columns[6], axis = 1, inplace = True, errors = 'ignore')

In [9]:
data

,0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,20
0,74,1,0.01,adam,avg,1000,None,relu,10,200,0.1,0.99,100,10,True,10,False,"[-10.0, 1000.0, -1.0]"


In [10]:
data.to_csv('ff.csv',index=False)

### now manually merge the files ff.csv and final.csv

In [11]:
experiements = pd.read_csv('Final.csv')

In [12]:
experiements.columns

Index(['ID', 'layers', 'tau', 'optimizer', 'advantage', 'max_timesteps',
       'gym_record', 'activation', 'batch_size', 'episodes', 'exploration',
       'gamma', 'hidden_size', 'train_repeat', 'display', 'min_train',
       'batch_norm', 'Rewardscheme', 'Details'],
      dtype='object')

In [13]:
experiements.sort_values(by=['ID'],inplace=True)

In [14]:
experiements['Details']=''

In [15]:
#experiements.set_value(experiements[experiements.ID==57].index[0],'Details','Vision 360')
experiements.set_value(experiements[experiements.ID==57].index[0],'Details','Vision 360');
experiements.set_value(experiements[experiements.ID==58].index[0],'Details','Static Env')
experiements.set_value(experiements[experiements.ID==59].index[0],'Details','Water walls')
experiements.set_value(experiements[experiements.ID==60].index[0],'Details','V360,water walls, static env');
experiements.set_value(experiements[experiements.ID==62].index[0],'Details','different SE');
experiements.set_value(experiements[experiements.ID==63].index[0],'Details','SE - water - 180v');
experiements.set_value(experiements[experiements.ID==65].index[0],'Details','SE - solid - 350v');
experiements.set_value(experiements[experiements.ID==66].index[0],'Details','Vision 90');
experiements.set_value(experiements[experiements.ID==71].index[0],'Details','Replication of Experiment E7');
experiements.set_value(experiements[experiements.ID==71].index[0],'Details','Replication of Experiment E7');
#experiements.set_value(experiements[experiements.ID==61].index[0],'Details','1000 Episode')

In [16]:
experiements.to_csv('Final.csv',index=False)